# Lesson 25: Keyword search and Vector Retrieval

## Introduction (5 minutes)

Welcome to our lesson on Keyword Search and Vector Retrieval. In this 60-minute session, we'll explore how to combine traditional keyword-based search techniques with modern vector retrieval methods to create more effective and efficient RAG systems.

## Lesson Objectives

By the end of this lesson, you will be able to:
1. Understand the principles of keyword search and vector retrieval
2. Implement basic keyword search using inverted indices
3. Combine keyword search with vector retrieval for hybrid search
4. Optimize hybrid search for RAG applications
5. Evaluate the performance of different search strategies

## 1. Principles of Keyword Search and Vector Retrieval (10 minutes)

### Keyword Search:
- Based on exact or partial matches of words
- Uses techniques like inverted indices for efficient retrieval
- Pros: Precise for specific terms; Cons: Misses semantic similarities

### Vector Retrieval:
- Based on semantic similarity of embeddings
- Uses techniques like nearest neighbor search in vector spaces
- Pros: Captures semantic meaning; Cons: Can be computationally expensive

Importance of combining both:
- Leverage strengths of both methods
- Improve relevance and recall in RAG systems
- Handle both specific queries and semantic understanding

## 2. Implementing Keyword Search (15 minutes)

Let's implement a basic keyword search using an inverted index:

In [ ]:
from collections import defaultdict
import re

class KeywordSearch:
    def __init__(self):
        self.inverted_index = defaultdict(list)
        self.documents = []

    def add_document(self, doc):
        doc_id = len(self.documents)
        self.documents.append(doc)
        
        words = re.findall(r'\w+', doc.lower())
        for word in set(words):  # Use set to count each word only once per document
            self.inverted_index[word].append(doc_id)

    def search(self, query, top_k=3):
        query_words = re.findall(r'\w+', query.lower())
        doc_scores = defaultdict(int)
        
        for word in query_words:
            for doc_id in self.inverted_index.get(word, []):
                doc_scores[doc_id] += 1
        
        sorted_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
        return [(self.documents[doc_id], score) for doc_id, score in sorted_docs[:top_k]]

# Usage
ks = KeywordSearch()
docs = [
    "Keyword search is based on exact matches.",
    "Vector retrieval uses semantic similarity.",
    "Hybrid search combines keywords and vectors.",
    "RAG systems benefit from both search methods."
]
for doc in docs:
    ks.add_document(doc)

results = ks.search("keyword vector search")
for doc, score in results:
    print(f"Score: {score}, Document: {doc}")

## 3. Combining Keyword Search with Vector Retrieval (20 minutes)

Now, let's implement a hybrid search system that combines keyword and vector retrieval:

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer

class HybridSearch:
    def __init__(self):
        self.keyword_search = KeywordSearch()
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.embeddings = []

    def add_document(self, doc):
        self.keyword_search.add_document(doc)
        self.embeddings.append(self.model.encode(doc))

    def vector_search(self, query, top_k=3):
        query_embedding = self.model.encode(query)
        similarities = np.dot(self.embeddings, query_embedding)
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        return [(self.keyword_search.documents[i], similarities[i]) for i in top_indices]

    def hybrid_search(self, query, top_k=3, alpha=0.5):
        keyword_results = self.keyword_search.search(query, top_k=len(self.keyword_search.documents))
        vector_results = self.vector_search(query, top_k=len(self.keyword_search.documents))
        
        combined_scores = defaultdict(float)
        for doc, score in keyword_results:
            combined_scores[doc] += alpha * score
        for doc, score in vector_results:
            combined_scores[doc] += (1 - alpha) * score
        
        sorted_results = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
        return sorted_results[:top_k]

# Usage
hs = HybridSearch()
docs = [
    "Keyword search is based on exact matches.",
    "Vector retrieval uses semantic similarity.",
    "Hybrid search combines keywords and vectors.",
    "RAG systems benefit from both search methods."
]
for doc in docs:
    hs.add_document(doc)

query = "How do RAG systems use different search techniques?"
results = hs.hybrid_search(query)
for doc, score in results:
    print(f"Score: {score:.4f}, Document: {doc}")

## 4. Optimizing Hybrid Search for RAG (15 minutes)

To optimize hybrid search for RAG applications, consider:

1. Adjusting the balance between keyword and vector search:
   - Tune the `alpha` parameter based on your specific use case
   - Use cross-validation to find the optimal balance

2. Implementing a two-stage retrieval process:
   - Use keyword search to filter initial candidates
   - Apply vector search on the filtered set for final ranking

3. Utilizing query expansion techniques:
   - Use synonyms or related terms to enhance keyword search
   - Generate multiple vector representations for a single query

Let's implement a two-stage retrieval process:

In [ ]:
class OptimizedHybridSearch(HybridSearch):
    def two_stage_search(self, query, first_stage_k=10, final_k=3):
        # First stage: Keyword search
        keyword_results = self.keyword_search.search(query, top_k=first_stage_k)
        candidate_docs = [doc for doc, _ in keyword_results]
        
        # Second stage: Vector search on candidates
        query_embedding = self.model.encode(query)
        candidate_embeddings = [self.model.encode(doc) for doc in candidate_docs]
        similarities = np.dot(candidate_embeddings, query_embedding)
        
        # Combine scores
        combined_scores = [(doc, 0.5 * kw_score + 0.5 * vec_score) 
                           for (doc, kw_score), vec_score in zip(keyword_results, similarities)]
        
        sorted_results = sorted(combined_scores, key=lambda x: x[1], reverse=True)
        return sorted_results[:final_k]

# Usage
ohs = OptimizedHybridSearch()
for doc in docs:
    ohs.add_document(doc)

results = ohs.two_stage_search(query)
for doc, score in results:
    print(f"Score: {score:.4f}, Document: {doc}")

## 5. Evaluating Search Strategies (5 minutes)

To evaluate different search strategies:

1. Prepare a test set with queries and relevant documents
2. Implement metrics like Precision, Recall, and Mean Average Precision (MAP)
3. Compare performance of keyword, vector, and hybrid search

Example evaluation code:

In [ ]:
def mean_average_precision(relevant_docs, retrieved_docs):
    if not relevant_docs:
        return 0.0
    
    score = 0.0
    num_hits = 0
    for i, doc in enumerate(retrieved_docs):
        if doc in relevant_docs:
            num_hits += 1
            score += num_hits / (i + 1)
    
    return score / len(relevant_docs)

# Evaluate different search methods
query = "RAG search techniques"
relevant_docs = {"RAG systems benefit from both search methods."}

keyword_results = ks.search(query)
vector_results = hs.vector_search(query)
hybrid_results = hs.hybrid_search(query)
optimized_results = ohs.two_stage_search(query)

print("Keyword Search MAP:", mean_average_precision(relevant_docs, [doc for doc, _ in keyword_results]))
print("Vector Search MAP:", mean_average_precision(relevant_docs, [doc for doc, _ in vector_results]))
print("Hybrid Search MAP:", mean_average_precision(relevant_docs, [doc for doc, _ in hybrid_results]))
print("Optimized Hybrid Search MAP:", mean_average_precision(relevant_docs, [doc for doc, _ in optimized_results]))

## Conclusion and Q&A (5 minutes)

In this lesson, we've explored the principles of keyword search and vector retrieval, and how to combine them for more effective RAG systems. We've implemented basic and advanced hybrid search methods and discussed optimization strategies. Remember, the best approach often depends on your specific use case and data.

Are there any questions about keyword search, vector retrieval, or their combination in RAG systems?

## Additional Resources

1. "Information Retrieval" by C. D. Manning, P. Raghavan, and H. Schütze: https://nlp.stanford.edu/IR-book/
2. "Neural Information Retrieval: A Literature Review" paper: https://arxiv.org/abs/1611.06792
3. Elasticsearch Learning to Rank plugin: https://elasticsearch-learning-to-rank.readthedocs.io/en/latest/
4. Facebook AI's FAISS library for efficient similarity search: https://github.com/facebookresearch/faiss

In our next lesson, we'll start working on a practical RAG project, applying the concepts we've learned so far.